In [5]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np   
import pandas as pd  

import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats
import pickle 

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn

In [6]:
RAW_DATA_PATH_X = '/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/X.csv'
RAW_DATA_PATH_y = '/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/y.csv'
X = pd.read_csv(RAW_DATA_PATH_X)
y = pd.read_csv(RAW_DATA_PATH_y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

In [8]:
def metrics_report(model,features,target):
    predictions = model.predict(features)
    acc = accuracy_score(target, predictions)  # to compute Accuracy
    recall = recall_score(target, predictions)  # to compute Recall
    precision = precision_score(target, predictions)  # to compute Precision
    f1 = f1_score(target, predictions)  # to compute F1-score

        # creating a dataframe of metrics
    df_metrics = pd.DataFrame({
            "Accuracy": acc,
            "Recall": recall,
            "Precision": precision,
            "F1 Score": f1,
            },index=[0],)

    return df_metrics

In [9]:
with mlflow.start_run():
  xgb_model = XGBClassifier(random_state=42, eval_metric='logloss')

  # parameters to choose from
  parameters = {
      "n_estimators": [10,30,50],
      "scale_pos_weight":[1,2,5],
      "subsample":[0.7,0.9,1],
      "learning_rate":[0.05, 0.1,0.2],
      "colsample_bytree":[0.7,0.9,1],
      "colsample_bylevel":[0.5,0.7,1]
  }

  # Type of scoring used to compare parameter combinations
  scorer = metrics.make_scorer(metrics.f1_score)

  #  grid search
  grid_obj = GridSearchCV(xgb_model, parameters,scoring=scorer,cv=3)
  grid_obj = grid_obj.fit(X_train, y_train)

  # Set the clf to the best combination of parameters
  xgb = grid_obj.best_estimator_

  # Fit the best algorithm to the data.
  xgb.fit(X_train, y_train)
  
  metrics_train = metrics_report(xgb,X_train,y_train)
  metrics_test = metrics_report(xgb,X_test,y_test)
  metrics_dataframe = pd.concat([metrics_train, metrics_test]).reset_index()
  metrics_dataframe['index'] = ['train','test']
  metrics_dataframe.set_index('index',inplace=True)


  mlflow.log_param("random_state", 42)
  mlflow.log_param("test_size", 0.3)
  #mlflow.log_params(parameters)
  mlflow.log_metrics(dict(metrics_dataframe.loc['train']))
  mlflow.log_metrics(dict(metrics_dataframe.loc['test']))
  mlflow.sklearn.log_model(xgb, "model")
  modelpath = "/Users/piyush/Desktop/dsml_Portfolio/visa/models/models-%s-%s" % ("model_XBG_Classifier", 1)
  mlflow.sklearn.save_model(xgb, modelpath)  
  